## Задание
Вам будет предоставлен набор данных о задачах, которые выполнялись в рамках строительства капитальных объектов на месторождениях нефти и газа. Набор содержит информацию о примерно 716 тысячах задач. Для каждой из задач доступна информация о ее названии в строительном плане, а также частично заданная информация об иерархии задач и обобщенных классах наименований, к которым относятся эти задачи (двух разных степеней детализации).

Используя эти данные, вам необходимо будет разработать семантическую модель, которая позволяла бы эффективно определять обобщенные классы для задач, у которых эта информация не представлена.

### Каждая задача описывается следующими атрибутами.
- work_name 
    * Текстовое название задачи в строительном плане (без предобработки).
- upper_works
    * Информация об иерархии названий объектов и блоков работ, в рамках которых выполнялась эта задача. Если задачи имеют одинаковое значение этого атрибута – это означает, что они выполнялись в рамках одного блока работ над одним объектом
(может быть пустым).
- generalized_work_class
    * Информация об обобщенном классе наименований работ, к
    которому относится задача (может быть пустым).
- global_work_class
    * Информация о самом высоком уровне обобщения названия задачи
(может быть пустым).


In [3]:
import pandas as pd
import numpy as np

## 1. Get the data and do spell correction

In [4]:
df = pd.read_csv('./lab2_oil_gas_field_construction_data.csv')
df.columns

Index(['work_name', 'generalized_work_class', 'global_work_class',
       'upper_works'],
      dtype='object')

In [5]:
# remove outliers 
train_data = df.loc[df['generalized_work_class'].notna()]
test_data = df.loc[df['generalized_work_class'].isna()]

cnt = train_data.generalized_work_class.value_counts().to_frame().reset_index()
cnt.columns

list_of_classes = cnt.loc[cnt['count'] > 500, 'generalized_work_class'].values
train_data = train_data[train_data['generalized_work_class'].isin(list_of_classes)]

In [6]:
train_data

,work_name,generalized_work_class,global_work_class,upper_works
3,монтад лестниц,Монтаж лестниц,Строительство зданий,NaN
7,кипиа,Монтаж приборов,Монтаж,NaN
9,электромонтажные работы 1.4.2,ПНР,ПНР,NaN
10,погружение свай св3 (108х6х5590),Погружение свай,Монтаж свай,NaN
13,монтаж стоек,Монтаж мк,Монтаж мк,"('Подготовка свай', 'Монтаж ограждения')"
...,...,...,...,...
716053,монтаж провода ас120/19,Монтаж кабеля,Монтаж электрики,NaN
716061,монтаж стоек,Монтаж мк,Монтаж мк,"('Подготовка свай', 'Монтаж ограждения')"
716062,монтаж трубопровода азота е5/1,Монтаж трубопровода,Прокладка трубопровода,NaN
716063,гравийно-песчаная подготовка пдн1 (коэф.уплотн...,Устройство песчаной подушки,Укладка,NaN


## 2. Correct spell mistakes 
- inify the form how the work name is written to use it for semantical class-tion later

In [8]:
from spellchecker import SpellChecker

russian = SpellChecker(language='ru', distance=1)

def correct_the_spelling(s):
    '''Corrects the spell mistakes in string'''
    new_string = ''
    for i in s:
        if i not in ['свай', 'балок', 'лестниц', 'т1', 'газа', 'бурение', 'пнр', 'мк']:
            i_corr = russian.correction(i)
            if i_corr != None:
                new_string = new_string + " " + i_corr
            else:
                new_string = new_string + " " + i
        else:
            new_string = new_string + " " + i
    return new_string


train_data['work_name_corr'] = train_data['work_name'].str.split()
train_data['work_name_corr'] = train_data['work_name_corr'].apply(correct_the_spelling)

test_data['work_name_corr'] = test_data['work_name'].str.split()
test_data['work_name_corr'] = test_data['work_name_corr'].apply(correct_the_spelling)

/var/folders/tt/rhnv8s9j45dfkl45r1y57vw80000gn/T/ipykernel_63076/938801483.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['work_name_corr'] = test_data['work_name'].str.split()


## 3. If a test data work_name is in previously accured mapping

- Automatically classify it in a class

In [ ]:
from pprint import pprint

target_text_dict = {}
unique_targets = train_data['generalized_work_class'].unique()

for target in unique_targets:
    target_texts = train_data[train_data['generalized_work_class'] == target]['work_name_corrected']
    unique_texts = target_texts.unique().tolist()
    target_text_dict[target] = unique_texts

pprint(target_text_dict.values())

In [ ]:
def map_labels(text):
    '''Maps work_name in test into classes accured in train data'''
    for label, unique_texts in target_text_dict.items():
        if text in unique_texts:
            return label
    return None

In [ ]:
test_data['predicted_labels'] = test_data['work_name_corr'].map(map_labels)

remain = test_data.loc[test_data['predicted_labels'].isna()]
print(f'Still unclassified in test data: {len(remain)} observations')

In [ ]:
train_data.to_csv('./Checkpoins/train_spell_correcred.csv')
test_data.to_csv('./Checkpoins/test_spell_correcred.csv')

## 4. The rest of unclaasified obs, we will claffify with ML model

```01_BERT_class.ipynb```